In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

# Optionally, print devices to confirm no GPU is being used
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

In [ ]:
pip install opencv-python fer pywhatkit

In [ ]:
import cv2
import numpy as np
import base64
from IPython.display import display, HTML, Javascript
from google.colab.output import eval_js
import matplotlib.pyplot as plt
from fer import FER

In [3]:
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
      async function takePhoto(quality) {
        const div = document.createElement('div');
        const captureButton = document.createElement('button');
        captureButton.textContent = 'Capture';
        div.appendChild(captureButton);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Adjust the iframe height to fit the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for the user to click the capture button.
        await new Promise((resolve) => captureButton.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
      }
      ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = base64.b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

In [ ]:
filename = take_photo()
print("Photo saved as:", filename)

In [ ]:
img = cv2.imread(filename)
if img is None:
    print("Error: Could not load the image. Please try again.")
else:
    # Display the captured image inline
    plt.figure(figsize=(6,4))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Captured Image")
    plt.show()

In [14]:
emotion_detector = FER(mtcnn=True)

In [ ]:
results = emotion_detector.detect_emotions(img)
if results:
    top_emotion, score = emotion_detector.top_emotion(img)
    print(f"Detected emotion: {top_emotion} with confidence {score:.2f}")

    # Map detected emotion to a song suggestion
    song_mapping = {
        'happy': 'Happy - Pharrell Williams',
        'sad': 'Someone Like You - Adele',
        'angry': 'Break Stuff - Limp Bizkit',
        'surprise': 'Surprise Yourself - Jack Garratt',
        'neutral': 'Best Day of My Life - American Authors'
    }
    song = song_mapping.get(top_emotion.lower(), song_mapping['neutral'])
    #print("Suggested song:", song)

    # Create a YouTube search URL for the song
    query = song.replace(" ", "+")
    url = f"https://www.youtube.com/results?search_query={query}"

    # Display a clickable link to the YouTube search results
    display(HTML(f"<a href='{url}' target='_blank'>Click here to play the song on YouTube</a>"))
else:
    print("No face or emotion detected in the image.")